<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/Desafio-NLP--Question%26Answer/Universal_Encoder_Q%26A_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Recuperação de perguntas e respostas utilizando codificador universal de frases multilíngue

##### https://www.tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa

Being able to automatically answer questions accurately remains a difficult problem in natural language processing. This dataset has everything you need to try your own hand at this task. Can you correctly generate the answer to questions given the Wikipedia article text the question was originally generated from?

Content:
There are three question files, one for each year of students: S08, S09, and S10, as well as 690,000 words worth of cleaned text from Wikipedia that was used to generate the questions.

The "questionanswerpairs.txt" files contain both the questions and answers. The columns in this file are as follows:

ArticleTitle is the name of the Wikipedia article from which questions and answers initially came.
Question is the question.
Answer is the answer.
DifficultyFromQuestioner is the prescribed difficulty rating for the question as given to the question-writer.
DifficultyFromAnswerer is a difficulty rating assigned by the individual who evaluated and answered the question, which may differ from the difficulty in field 4.
ArticleFile is the name of the file with the relevant article

Questions that were judged to be poor were discarded from this data set.
There are frequently multiple lines with the same question, which appear if those questions were answered by multiple individuals. https://www.kaggle.com/rtatman/questionanswer-dataset

### Solução

##### Cada resposta no dataset QA (Sxx_question_answer_pairs.txt) e seu contexto (o texto em torno da frase) em seu artigo correspondente (Sxx_setx_ax.txt.clean), são codificados em embeddings de alta dimensão com o response_encoder. Esses embeddings são armazenados em um índice construído usando a biblioteca simpleneighbors para recuperação de perguntas e respostas.

In [1]:
!python -m pip uninstall -y spacy

Found existing installation: spacy 3.0.6
Uninstalling spacy-3.0.6:
  Successfully uninstalled spacy-3.0.6


In [2]:
!python -m pip install -q tensorflow_text
!python -m pip install -q simpleneighbors[annoy]
!python -m pip install -U pip setuptools wheel
!python -m pip install -U spacy[cuda102]
!python -m spacy download en_core_web_lg

  Using cached spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl (12.8 MB)
2021-06-08 21:20:08.496740: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8 MB 12 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import spacy
print(spacy.__version__)
nlp = spacy.load('en_core_web_lg')

3.0.6


In [4]:
import pandas as pd
import glob
import re
import string
import os
import tqdm.notebook as tq
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
from simpleneighbors import SimpleNeighbors
import random
import pprint
from IPython.display import HTML, display

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/desafio 3/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Extrair o **contexto** dos artigos Sxx_setx_ax.txt.clean

In [6]:
files = glob.glob('/content/drive/My Drive/desafio 3/text_data/*.clean')

list_text = []
list_file = []

for file in files:
  with open(file, 'r', encoding = 'utf-8', errors='ignore') as f:
    text = f.read()
  list_text.append(re.sub(r'\n+', ' ', text).strip())
  list_file.append((os.path.basename(file)).split('.')[0])
df_context = pd.DataFrame({'Context': list_text, 'ArticleFile': list_file})

In [7]:
#df_context['context_doc'] = [nlp(text) for text in df_context.Context]

#### Criar dataframes de QA Sxx_question_answer_pairs.txt

In [8]:
df_08 = pd.read_table('S08_question_answer_pairs.txt')
df_09 = pd.read_table('S09_question_answer_pairs.txt')
df_10 = pd.read_table('S10_question_answer_pairs.txt', engine = 'python', error_bad_lines = False)

#df_qa = pd.concat([df_08, df_09, df_10], axis=0, ignore_index=True)

Skipping line 765: '	' expected after '"'
Skipping line 876: '	' expected after '"'
Skipping line 1219: '	' expected after '"'


In [9]:
df_08.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)
df_09.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)
df_10.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)

In [10]:
df_08.dropna(inplace=True)
df_09.dropna(inplace=True)
df_10.dropna(inplace=True)
print('-' * 15, df_08.isna().sum(), sep='\n')
print('-' * 15, df_09.isna().sum(), sep='\n')
print('-' * 15, df_10.isna().sum(), sep='\n')

---------------
Question       0
Answer         0
ArticleFile    0
dtype: int64
---------------
Question       0
Answer         0
ArticleFile    0
dtype: int64
---------------
Question       0
Answer         0
ArticleFile    0
dtype: int64


In [11]:
# Limpar coluna "Answer"
def strip_last_punctuation(s):
  if s and s[-1] in string.punctuation:
    return s[:-1].strip()
  else:
    return s.strip()


df_08['answer_clean'] = df_08['Answer'].str.lower().map(strip_last_punctuation)
df_09['answer_clean'] = df_09['Answer'].str.lower().map(strip_last_punctuation)
df_10['answer_clean'] = df_10['Answer'].str.lower().map(strip_last_punctuation)

# Remove os dados faltantes da base de treino
df_08.dropna(inplace=True)
df_09.dropna(inplace=True)
df_10.dropna(inplace=True)
print('-' * 15, df_08.isna().sum(), sep='\n')
print('-' * 15, df_09.isna().sum(), sep='\n')
print('-' * 15, df_10.isna().sum(), sep='\n')

---------------
Question        0
Answer          0
ArticleFile     0
answer_clean    0
dtype: int64
---------------
Question        0
Answer          0
ArticleFile     0
answer_clean    0
dtype: int64
---------------
Question        0
Answer          0
ArticleFile     0
answer_clean    0
dtype: int64


In [12]:
df_USE08 = df_08.merge(df_context, on='ArticleFile')
df_USE09 = df_09.merge(df_context, on='ArticleFile')
df_USE10 = df_10.merge(df_context, on='ArticleFile')

In [13]:
print('-' * 15, df_USE08.shape, sep='\n')
print('-' * 15, df_USE09.shape, sep='\n')
print('-' * 15, df_USE10.shape, sep='\n')

---------------
(1473, 5)
---------------
(725, 5)
---------------
(1219, 5)


In [16]:
df_USE08.drop_duplicates(subset=['answer_clean', 'Question'], keep='last', inplace = True)
df_USE09.drop_duplicates(subset=['answer_clean', 'Question'], keep='last', inplace = True)
df_USE10.drop_duplicates(subset=['answer_clean', 'Question'], keep='last', inplace = True)

print('-' * 15, df_USE08.shape, sep='\n')
print('-' * 15, df_USE09.shape, sep='\n')
print('-' * 15, df_USE10.shape, sep='\n')

---------------
(1148, 5)
---------------
(590, 5)
---------------
(1045, 5)


### Carregar modelo **Universal Encoder Q&A Model** do tensorflow hub

In [15]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3"
model = hub.load(module_url)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


### Funções para testar o modelo - > decodificar a questão usando question_encoder e o embedding da questão é usado para consultar o índice simpleneighbors.

In [21]:
def output_with_highlight(text, highlight):
  output = "<li> "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += '<b>'+text[i:i+len(highlight)]+'</b>'
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "</li>\n"

def display_nearest_neighbors(query_text, answer_text=None):
    query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
    search_results = index.nearest(query_embedding, n=num_results)

    if answer_text:
      result_md = '''
      <p>Random Question:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      <p>Answer:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      ''' % (query_text , answer_text)
    else:
      result_md = '''
      <p>Question:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      ''' % query_text

    result_md += '''
      <p>Retrieved sentences :
      <ol>
    '''

    if answer_text:
      for s in search_results:
        result_md += output_with_highlight(s, answer_text)
    else:
      for s in search_results:
        result_md += '<li>' + s + '</li>\n'

    result_md += "</ol>"
    display(HTML(result_md))

### S08_question_answer_pairs.txt

#### Tranformar o dataframe em listas answer_context_S08 e question_S08

In [17]:
answer_context_S08 = []
question_S08 = []

for index, r in df_USE08.iterrows():
  answer_context_S08.append([r['answer_clean'],r['Context']])
  question_S08.append(r['Question'])

In [18]:
answer_context_S08[100]

['yes',
 'Calvin Coolidge   John Calvin Coolidge, Jr. (July 4 1872   January 5 1933), more commonly known as Calvin Coolidge, was the thirtieth President of the United States (1923–1929).  A lawyer from Vermont, Coolidge worked his way up the ladder of Massachusetts state politics, eventually becoming governor of that state. His actions during the Boston Police Strike of 1919 thrust him into the national spotlight. Soon after, he was elected as the twenty-ninth Vice President in 1920 and succeeded to the Presidency upon the death of Warren G. Harding. Elected in his own right in 1924, he gained a reputation as a small-government conservative. In many ways Coolidge\'s style of governance was a throwback to the passive presidency of the nineteenth century. Sobel, 14  He restored public confidence in the White House after the scandals of his predecessor\'s administration, and left office with considerable popularity. McCoy, 420–421; Greenberg, 49–53  As his biographer later put it, "he em

#### Computar embeddings e criar o simpleneighbors index

In [19]:
batch_size = 100

encodings = model.signatures['response_encoder'](  input=tf.constant([answer_context_S08[0][0]]), context=tf.constant([answer_context_S08[0][1]]))
index = SimpleNeighbors(len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(answer_context_S08))
slices = zip(*(iter(answer_context_S08),) * batch_size)
num_batches = int(len(answer_context_S08) / batch_size)
for s in tq.tqdm_notebook(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(answer_context_S08))

Computing embeddings for 1148 sentences



simpleneighbors index for 1148 sentences built.


#### Testar o modelo

In [22]:
#@title ###### Acessar os "vizinhos próximos" **nearest neighbors** para perguntas escolhidas randomicamente
num_results = 5 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(question_S08)
display_nearest_neighbors(query)

### S09_question_answer_pairs.txt

#### Tranformar o dataframe em listas answer_context_S09 e question_S09

In [23]:
answer_context_S09 = []
question_S09 = []

for index, r in df_USE09.iterrows():
  answer_context_S09.append((r['answer_clean'].lower(),r['Context'].lower()))
  question_S09.append(r['Question'].lower())

#### Computar embeddings e criar o simpleneighbors index

In [24]:
batch_size = 100

encodings = model.signatures['response_encoder'](  input=tf.constant([answer_context_S09[0][0]]), context=tf.constant([answer_context_S09[0][1]]))
index = SimpleNeighbors(len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(answer_context_S09))
slices = zip(*(iter(answer_context_S09),) * batch_size)
num_batches = int(len(answer_context_S09) / batch_size)
for s in tq.tqdm_notebook(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(answer_context_S09))

Computing embeddings for 590 sentences



simpleneighbors index for 590 sentences built.


In [25]:
#@title ###### Acessar os "vizinhos próximos" **nearest neighbors** para perguntas escolhidas randomicamente
num_results = 5 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(question_S09)
display_nearest_neighbors(query)

### S10_question_answer_pairs.txt

#### Tranformar o dataframe em listas answer_context_S10 e question_S10

In [26]:
answer_context_S10 = []
question_S10 = []

for index, r in df_USE10.iterrows():
  answer_context_S10.append((r['Answer'].lower(),r['Context'].lower()))
  question_S10.append(r['Question'])

#### Computar embeddings e criar o **simpleneighbors** index

In [27]:
batch_size = 100

encodings = model.signatures['response_encoder'](  input=tf.constant([answer_context_S10[0][0]]), context=tf.constant([answer_context_S10[0][1]]))
index = SimpleNeighbors(len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(answer_context_S10))
slices = zip(*(iter(answer_context_S10),) * batch_size)
num_batches = int(len(answer_context_S10) / batch_size)
for s in tq.tqdm_notebook(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(answer_context_S10))


Computing embeddings for 1045 sentences



simpleneighbors index for 1045 sentences built.


#### Testar o modelo

In [28]:
#@title ###### Acessar os "vizinhos próximos" **nearest neighbors** para perguntas escolhidas randomicamente
num_results = 5 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(question_S10)
display_nearest_neighbors(query)